**Sarcasm Detection Case Study**
Loading sarcasm detection dataset from this source - it is high quality data from The Onion which is a sarcastic news site. Idea is to use this to test pretrained models from Hugging Face for sarcasm detection and compare.
# https://github.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection

url = 'https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json'


In [ ]:
import requests
import pandas as pd
import json

# Replace the URL below with the raw URL of your JSON file
url = 'https://raw.githubusercontent.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection/master/Sarcasm_Headlines_Dataset.json'

# Fetch the file from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Split the response text by newlines
    lines = response.text.splitlines()

    # Parse each line as a JSON object and append to a list
    data = [json.loads(line) for line in lines]

    # Create a DataFrame from the list
    df = pd.DataFrame(data)

    # Display the first few rows of the DataFrame
    print(df.head())
else:
    print(f"Failed to retrieve the file: {response.status_code}")


   is_sarcastic                                           headline  \
0             1  thirtysomething scientists unveil doomsday clo...   
1             0  dem rep. totally nails why congress is falling...   
2             0  eat your veggies: 9 deliciously different recipes   
3             1  inclement weather prevents liar from getting t...   
4             1  mother comes pretty close to using word 'strea...   

                                        article_link  
0  https://www.theonion.com/thirtysomething-scien...  
1  https://www.huffingtonpost.com/entry/donna-edw...  
2  https://www.huffingtonpost.com/entry/eat-your-...  
3  https://local.theonion.com/inclement-weather-p...  
4  https://www.theonion.com/mother-comes-pretty-c...  


In [ ]:
pd.set_option('display.max_colwidth', None)


In [ ]:
df

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clock of hair loss,https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205
1,0,"dem rep. totally nails why congress is falling short on gender, racial equality",https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html
3,1,inclement weather prevents liar from getting to work,https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031
4,1,mother comes pretty close to using word 'streaming' correctly,https://www.theonion.com/mother-comes-pretty-close-to-using-word-streaming-cor-1819575546
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-rosh-hashasha-or-something-1819564013
28615,1,internal affairs investigator disappointed conspiracy doesn't go all the way to the top,https://local.theonion.com/internal-affairs-investigator-disappointed-conspiracy-d-1819568967
28616,0,the most beautiful acceptance speech this week came from a queer korean,https://www.huffingtonpost.com/entry/andrew-ahn-independent-spirit-awards_us_58b44741e4b060480e0a2c30
28617,1,mars probe destroyed by orbiting spielberg-gates space palace,https://www.theonion.com/mars-probe-destroyed-by-orbiting-spielberg-gates-space-1819564363


In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix
# Load the model and tokenizer for TensorFlow
model_name = "helinivan/english-sarcasm-detector"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)





config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Function to perform sarcasm detection
def detect_sarcasm(sentence):
    inputs = tokenizer(sentence, return_tensors="tf")
    outputs = model(inputs)

    # Process the model output
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1)

    # Convert predicted class to label
    labels = ['Not Sarcasm', 'Sarcasm']
    predicted_label = tf.gather(labels, predicted_class)

    return predicted_label.numpy()[0], probabilities.numpy()[0][predicted_class]
    # return predicted_class.numpy()[0]

In [ ]:
# Example sentence
sentence = "Oh great, another Monday."

# Detect sarcasm
# label, confidence = detect_sarcasm(sentence)
# label, confidence

detect_sarcasm(sentence)

0

In [ ]:
df

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clock of hair loss,https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205
1,0,"dem rep. totally nails why congress is falling short on gender, racial equality",https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html
3,1,inclement weather prevents liar from getting to work,https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031
4,1,mother comes pretty close to using word 'streaming' correctly,https://www.theonion.com/mother-comes-pretty-close-to-using-word-streaming-cor-1819575546
...,...,...,...
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-rosh-hashasha-or-something-1819564013
28615,1,internal affairs investigator disappointed conspiracy doesn't go all the way to the top,https://local.theonion.com/internal-affairs-investigator-disappointed-conspiracy-d-1819568967
28616,0,the most beautiful acceptance speech this week came from a queer korean,https://www.huffingtonpost.com/entry/andrew-ahn-independent-spirit-awards_us_58b44741e4b060480e0a2c30
28617,1,mars probe destroyed by orbiting spielberg-gates space palace,https://www.theonion.com/mars-probe-destroyed-by-orbiting-spielberg-gates-space-1819564363


In [ ]:
df['is_sarcastic'].value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

In [ ]:
# Balancing the dataset
balanced_df = df.groupby('is_sarcastic').sample(n=1000, random_state=1)  # 250 from each class to get 500 total

# Display the balanced sample
balanced_df['is_sarcastic'].value_counts(), balanced_df.head()

(0    1000
 1    1000
 Name: is_sarcastic, dtype: int64,
        is_sarcastic  \
 573               0   
 2162              0   
 19879             0   
 4331              0   
 25510             0   
 
                                                                         headline  \
 573               will ferrell breaks into the kardashian house for a good cause   
 2162           black twitter is freaking out over harriet tubman on the $20 bill   
 19879  paul ryan says he and steve bannon are 'different kinds of conservatives'   
 4331                           facebook outreach tool ignores black lives matter   
 25510                                      ten states with the most student debt   
 
                                                                                                                             article_link  
 573      https://www.huffingtonpost.com/entry/will-ferrell-breaks-into-the-kardashian-house-for-a-good-cause_us_5a297f89e4b03ece0300e845  
 2162

In [ ]:
balanced_df

,is_sarcastic,headline,article_link
573,0,will ferrell breaks into the kardashian house for a good cause,https://www.huffingtonpost.com/entry/will-ferrell-breaks-into-the-kardashian-house-for-a-good-cause_us_5a297f89e4b03ece0300e845
2162,0,black twitter is freaking out over harriet tubman on the $20 bill,https://www.huffingtonpost.com/entry/black-twitter-is-freaking-out-over-harriet-tubman-on-the-20-bill_us_5717ad01e4b024dae4f0a980
19879,0,paul ryan says he and steve bannon are 'different kinds of conservatives',https://www.huffingtonpost.com/entry/paul-ryan-steve-bannon_us_589bac39e4b0c1284f2a91c4
4331,0,facebook outreach tool ignores black lives matter,https://www.huffingtonpost.comhttps://theintercept.com/2016/06/09/facebook-outreach-tool-ignores-black-lives-matter/
25510,0,ten states with the most student debt,https://www.huffingtonpost.com/entry/ten-states-with-the-most-_0_n_5561627.html
...,...,...,...
27825,1,valentine's day coming a little early in relationship,https://local.theonion.com/valentines-day-coming-a-little-early-in-relationship-1819566329
988,1,thousands dead in indonesia again,https://www.theonion.com/thousands-dead-in-indonesia-again-1819564455
26380,1,chinese tv show canceled after drawing only 180 million viewers,https://www.theonion.com/chinese-tv-show-canceled-after-drawing-only-180-million-1819569946
9972,1,"gop officials urge calmer, more reasonable death threats toward kavanaugh accuser",https://politics.theonion.com/gop-officials-urge-calmer-more-reasonable-death-threat-1829209510


In [ ]:
# Shuffling the balanced dataset
df1 = shuffled_df = balanced_df.sample(frac=1, random_state=1).reset_index(drop=True)

# Display the first few rows of the shuffled dataframe
shuffled_df.head()

,is_sarcastic,headline,article_link
0,0,arkansas gets permission to enforce voter id law in primaries,https://www.huffingtonpost.com/entry/arkansas-voter-id-law-primaries_us_5aea305ee4b00f70f0eedcb1
1,1,bannon forced to cancel 'muscle & fitness' cover shoot to testify before grand jury,https://politics.theonion.com/bannon-forced-to-cancel-muscle-fitness-cover-shoot-to-1822127548
2,1,buick introduces new self-buying car,https://www.theonion.com/buick-introduces-new-self-buying-car-1820796591
3,1,new country-music video has look of 1991 rock video,https://entertainment.theonion.com/new-country-music-video-has-look-of-1991-rock-video-1819565902
4,1,stormy daniels '60 minutes' interview leads to spike in pornhub searches for anderson cooper,https://politics.theonion.com/stormy-daniels-60-minutes-interview-leads-to-spike-in-1824075371


In [ ]:
from tqdm import tqdm
tqdm.pandas()

df1['predicted'] = df1['headline'].progress_apply(detect_sarcasm)

# Apply the function to the DataFrame
# df['predicted'] = df['headline'].apply(detect_sarcasm)



100%|██████████| 2000/2000 [06:31<00:00,  5.11it/s]


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(df1['is_sarcastic'], df1['predicted'])
print(f"Accuracy: {accuracy:.2f}")

# Generate confusion matrix
conf_matrix = confusion_matrix(df1['is_sarcastic'], df1['predicted'])
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.94
Confusion Matrix:
[[986  14]
 [103 897]]


In [ ]:
df1

,is_sarcastic,headline,article_link,predicted
0,0,arkansas gets permission to enforce voter id law in primaries,https://www.huffingtonpost.com/entry/arkansas-voter-id-law-primaries_us_5aea305ee4b00f70f0eedcb1,0
1,1,bannon forced to cancel 'muscle & fitness' cover shoot to testify before grand jury,https://politics.theonion.com/bannon-forced-to-cancel-muscle-fitness-cover-shoot-to-1822127548,1
2,1,buick introduces new self-buying car,https://www.theonion.com/buick-introduces-new-self-buying-car-1820796591,1
3,1,new country-music video has look of 1991 rock video,https://entertainment.theonion.com/new-country-music-video-has-look-of-1991-rock-video-1819565902,1
4,1,stormy daniels '60 minutes' interview leads to spike in pornhub searches for anderson cooper,https://politics.theonion.com/stormy-daniels-60-minutes-interview-leads-to-spike-in-1824075371,0
...,...,...,...,...
1995,1,man spends entire marketing meeting nodding,https://www.theonion.com/man-spends-entire-marketing-meeting-nodding-1819575271,1
1996,1,chuck grassley scratches 'christine blasey's a slut' into senate bathroom stall,https://politics.theonion.com/chuck-grassley-scratches-christine-blasey-s-a-slut-in-1829460008,0
1997,1,area man thankful to be single during golden age of television,https://local.theonion.com/area-man-thankful-to-be-single-during-golden-age-of-tel-1829909473,1
1998,0,nordstrom just low-key dropped a huge fall sale,https://www.huffingtonpost.com/entry/the-best-of-nordstroms-fall-sale_us_59fb2a49e4b0b0c7fa3880bf,0


In [ ]:
detect_sarcasm("Prequel Depicts Young Willy Wonka Using Rich Father’s Investment To Buy Already-Successful Chocolate Factory")

# https://www.theonion.com/prequel-depicts-young-willy-wonka-using-rich-father-s-i-1851049152
# https://www.msn.com/en-in/sports/other/india-women-create-history-with-410-runs-on-day-1-of-only-test-against-england/ar-AA1lvxIx?ocid=hpmsn&cvid=db426707b24349c4aeae6501105d9e2e&ei=22

(b'Sarcasm', array([0.96305203], dtype=float32))

In [ ]:
detect_sarcasm("India women create history with 410 runs on day 1 of only Test against England")


(b'Not Sarcasm', array([0.85771537], dtype=float32))

In [ ]:
detect_sarcasm("Oh great, here comes another Monday")

(b'Not Sarcasm', array([0.84545916], dtype=float32))